# Detecção de Raças de Cachorros utilizando Redes Neurais

## Importações

### Importando biblioteca

In [58]:

from sys import platform
import os
import pandas as pd
from PIL import Image
import numpy as np
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf

### Importando Dataset

In [59]:
if platform == "linux" or platform == "linux2":
    folder_path = os.getcwd() + "/" + "low-resolution"
elif platform == "win32":
   folder_path = os.getcwd() + "\\" + "low-resolution"


list_images = []

for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)
    
    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            
            file_path = os.path.join(subfolder_path, file)
            image_about = {}
            breed = subfolder.split("-")[-1]
            image_about["breed"] = breed
            image_about["filename"] = file
            image_about["path_file"] = file_path
            list_images.append(image_about)

df_images = pd.DataFrame.from_records(list_images)

### Pré-processamento

In [71]:
if platform == "linux" or platform == "linux2":
    os_path = "/" 
elif platform == "win32":
   os_path = "\\"

folder_path = os.getcwd() + os_path + "low-resolution"
slot_size = 20

#Retorna uma tupla com os path de cada imagem e o target randomizados
def load_path_of_dataset():
    arquivos = os.listdir(folder_path)
    subpastas = [subpasta for subpasta in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, subpasta))]
    imagens_path = []
    labels = []
    for subpasta in subpastas:
        subpasta_path = os.path.join(folder_path, subpasta)
        imagens = os.listdir(subpasta_path)
        imagens_path.extend([subpasta + os_path + s for s in imagens])
        label = subpasta.split("-")[2]
        labels.extend([label for s in imagens])
    # minha_lista = list(zip(imagens_path, labels))
    # random.shuffle(minha_lista)
    return imagens_path, labels

#função responsavel em realizar a divisão em slots
#recebe como parametro a tupla da função load_path_of_dataset, e o numero da interação para carregar o dataframe
#retorna uma tupla menor, e o numero máximo de interações posiveis até o fim do dataset
def slot_loading(interacao, imagens_amostradas, imagens_labels):
    if interacao <= 0:
        raise ValueError('O valor da interação, deve ser maior que zero ! ')
    elif interacao > (interacao * len(imagens_amostradas)):
        raise ValueError('O valor da interação, é muito grande !')
    
    start_index = (interacao - 1) * slot_size
    end_index = interacao * slot_size
    if end_index > len(imagens_amostradas):
        return imagens_amostradas[start_index:], imagens_labels[start_index:]

    return imagens_amostradas[start_index:end_index], imagens_labels[start_index:end_index], len(imagens_amostradas) // slot_size

#Recebe como parametro uma tupla
#retorna um dataset já pré-processado e formatados
def get_images(imagens_amostradas):
    dataset = pd.DataFrame(columns=['Image', 'Label'])
    img_size_height = 300
    img_size_width = 400
    for imagem_nome in imagens_amostradas:
        try:
            # extensao = os.path.splitext(imagem_nome)[1].lower()
            imagem_path = os.path.join(folder_path, imagem_nome)
            imagem = Image.open(imagem_path)
            imagem = imagem.resize((img_size_width, img_size_height))  # Redimensiona para um tamanho comum
            imagem_array = np.array(imagem) / 255.0  # Normaliza dividindo por 255
            dataset.loc[len(dataset)] = [imagem_array, imagem_nome[1]]
        except Exception as error:
            print("ocorreu um erro com imagem_path: ", imagem_path, " imagem_nome " + imagem_nome," imagem_nome[0] " + imagem_nome[0], " imagens_amostradas: " , imagens_amostradas)

    return dataset;


#### Exemplo de utilização

In [72]:
X, y = load_path_of_dataset()
x_new, y_new, interations = slot_loading(1, X, y)
# slot_path_dataset, interations = slot_loading(1,path_dataset)
dataset = get_images(x_new);
print(dataset)

['201-n000024-schipperke/n110371.jpg', '201-n000024-schipperke/n110490.jpg', '201-n000024-schipperke/n110400.jpg', '201-n000024-schipperke/n110503.jpg', '201-n000024-schipperke/n110488.jpg', '201-n000024-schipperke/n110373.jpg', '201-n000024-schipperke/n110355.jpg', '201-n000024-schipperke/n110468.jpg', '201-n000024-schipperke/n110363.jpg', '201-n000024-schipperke/n110446.jpg', '201-n000024-schipperke/n110421.jpg', '201-n000024-schipperke/n110428.jpg', '201-n000024-schipperke/n110544.jpg', '201-n000024-schipperke/n110536.jpg', '201-n000024-schipperke/n110462.jpg', '201-n000024-schipperke/n110398.jpg', '201-n000024-schipperke/n110481.jpg', '201-n000024-schipperke/n110518.jpg', '201-n000024-schipperke/n110426.jpg', '201-n000024-schipperke/n110394.jpg', '201-n000024-schipperke/n110519.jpg', '201-n000024-schipperke/n110418.jpg', '201-n000024-schipperke/n110538.jpg', '201-n000024-schipperke/n110399.jpg', '201-n000024-schipperke/n110359.jpg', '201-n000024-schipperke/n110390.jpg', '201-n00002

## Separação do Conjunto de teste e treino

In [73]:
# Sua tupla
X, y = load_path_of_dataset()

# Separando 70% para treino e 30% para teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

# Exibindo as dimensões dos conjuntos de treino e teste
print("Dimensões do conjunto de treino:", len(X_treino), len(y_treino))
print("Dimensões do conjunto de teste:", len(X_teste), len(y_teste))

Dimensões do conjunto de treino: 49302 49302
Dimensões do conjunto de teste: 21130 21130


## Criação do modelo

In [74]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
# Model Architecture
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64 , kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))
model.add(Conv2D(filters=128 , kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(133,activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 224, 224, 16)      208       
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 112, 112, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 112, 112, 32)      2080      
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 56, 56, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_18 (Conv2D)          (None, 56, 56, 64)        8256      
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 28, 28, 64)       

#### Treinamento em batches 

In [75]:

# X = [item[0] for item in path_dataset]  # Recursos (features) da tupla
# y = [item[1] for item in path_dataset]   # Rótulos (labels) da tupla

# Treinar o modelo
x_new, y_new, i = slot_loading(1, X_treino, y_treino)
# model.fit(x_new, y_new)

for o in range(1, i):
    print( o," of " ,i)
    x_new, y_new = None, None
    x_new, y_new, i = slot_loading(o, X_treino, y_treino)
    x_new = get_images(x_new)
    print(x_new)
    print(y_new)
    print(i)
    model.fit(x_new , y_new, 
               shuffle = True,
               batch_size = 20,
               epochs = 25,
               verbose = 1)
# # Avaliar o modelo
# score = model.score(dados_teste, rotulos_teste)
# print("Acurácia do modelo:", score)




1  of  2465
                                                Image Label
0   [[[0.37254901960784315, 0.5843137254901961, 0....     7
1   [[[0.3568627450980392, 0.3411764705882353, 0.2...     0
2   [[[0.8274509803921568, 0.8313725490196079, 0.8...     3
3   [[[0.9686274509803922, 0.9294117647058824, 0.8...     9
4   [[[0.09019607843137255, 0.10980392156862745, 0...     1
5   [[[0.41568627450980394, 0.40784313725490196, 0...     3
6   [[[0.7764705882352941, 0.7725490196078432, 0.5...     3
7   [[[0.396078431372549, 0.4, 0.41568627450980394...     1
8   [[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...     1
9   [[[0.35294117647058826, 0.24313725490196078, 0...     5
10  [[[0.7529411764705882, 0.8274509803921568, 0.9...     0
11  [[[0.35294117647058826, 0.3215686274509804, 0....     5
12  [[[0.08235294117647059, 0.1450980392156863, 0....     0
13  [[[0.9333333333333333, 0.9607843137254902, 0.9...     3
14  [[[0.0, 0.00784313725490196, 0.019607843137254...     0
15  [[[0.28627450980392155, 

ValueError: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, (<class 'list'> containing values of types {"<class 'str'>"})